### Tensor的维度变换
常用于 对输入数据的尺寸调整，分为四种情况：
1. 矩阵尺度变换 -- view / reshape 函数
2. 维度通道挤压与展开 -- squeeze / unsqueeze 函数
3. 维度通道数的扩展 -- expand / repeat 函数
4. 维度位置的交换 -- transpose / permute 函数

**需要注意的问题是，数据尺寸调整后再复原，若通道顺序有变换，则会出现数据被破坏，信息被污染的情况**

### 广播机制
pytorch中使用的广播机制，允许多维矩阵与不同维度的矩阵或向量直接运算  
使用前提：首先是以末端维度为开始对齐;两个矩阵中，低维矩阵的匹配维度的通道数，必须与高维矩阵通道数相同或者是1通道。  
即 A -->[4,3,28,28], B --> [3,1,1]，C -->[2,28,28], D -->[1]  
首先末端对齐，再判断通道数是否相同或为1   
A -->[4,3,28,28]  
B -->......[3,1,1]  
C -->..[2,28,28]  
D -->............[1]   
可见 A能和 B,D运算，不能与C运算

In [1]:
import torch

In [47]:
# img -- > [B,C,H,W]
img = torch.rand(4,3,28,28)

#### reshape 与view 完全相同

In [11]:
# 想拉直图像像素作为特征 【28,28】 --> 28*28
img.view(img.size(0),img.size(1),-1).shape

torch.Size([4, 3, 784])

In [14]:
# reshape 函数与 view 函数通用，作用一样
feature = img.reshape(4,3,-1)
feature.shape

torch.Size([4, 3, 784])

#### unsqueeze 与squeeze 是一对对称函数
展开与压缩

In [30]:
# 给特征加一个偏差，这就要求bias的尺寸相匹配
bias = torch.randn(3)
print(bias.shape)
bias = bias.unsqueeze(0).unsqueeze(-1)
print(bias.shape)
bias.squeeze().shape

torch.Size([3])
torch.Size([1, 3, 1])


torch.Size([3])

In [34]:
feature = feature + bias

#### 解释 expand 与repeat 的不同
repeat 是执行是复制操作数据，占用内存空间，且repeat（1,3,1）里面值的含义是复制多少次，如例子就是第二通道，复制3次，不管原本有多少通道数据。  
expand 只有在用到该数据时才执行扩展，但只能针对通道数量是1的维度操作，expand（4,1,1），就是第一通道拓展成4个通道。

In [40]:
gamma = torch.randn(1).squeeze(0).squeeze(-1)
gamma.expand(1,3,1)

tensor([[[-1.3170],
         [-1.3170],
         [-1.3170]]])

In [41]:
gamma.repeat(1,3,1)

tensor([[[-1.3170],
         [-1.3170],
         [-1.3170]]])

In [43]:
img.shape

torch.Size([4, 3, 28, 28])

In [48]:
# img [B,C,H,W]--> [B,W,H,C] --> [B,n_elem] --> [B,C,H,W]
img2 = img.transpose(1,3).contiguous().view(img.size(0),-1).view(4,3,28,28)
img2.shape

torch.Size([4, 3, 28, 28])

注意这里，虽然通道数一样，但顺序变换后，数据就对不上了，输出为False

In [49]:
torch.all(torch.eq(img,img2))

tensor(0, dtype=torch.uint8)

#### permute 与 transpose
推荐使用permute，直接明了指定通道位置，transpose每次只能变换一次通道

#### 解释congtiguous（）的作用
当transpose（）函数打乱数据存储时，需要用contiguous（）来重新连接数据

In [51]:
img3 = img.transpose(1,3).contiguous().view(4,-1).view(4,28,28,3).transpose(1,3)
img3.shape

torch.Size([4, 3, 28, 28])

In [52]:
torch.all(torch.eq(img,img3))

tensor(1, dtype=torch.uint8)

In [56]:
img4 = img.permute(0,2,3,1)
img4.shape

torch.Size([4, 28, 28, 3])

#### 广播机制
意义，减少维度变换函数的调用和数据内存的占用

In [61]:
A = torch.randn(4,3,28,28)

In [62]:
B = torch.rand(3,1,1)
C = torch.rand(2,28,28)
D = torch.rand(1)

In [64]:
(A + B).shape

torch.Size([4, 3, 28, 28])

In [65]:
(A+C).shape

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [66]:
(A+D).shape

torch.Size([4, 3, 28, 28])